# Challenge 5: Multi-Source, Multi-Agent
## Introduction

In this part of the challenge you will add another source of data (structured) to the solution.

<div class="alert alert-block alert-warning">
Be Sure you populated correctly the `.env` file for the AZURE_SEARCH_INDEX. 
We are using <a href="https://pypi.org/project/python-dotenv/">python-dotenv</a> to manage our environment variables. It will also make things easier when deploying the application in Azure.

</div>

### Importing the libraries

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

### Create Environment Variables

**Important:** Make sure you update your `.env` file.

In [ ]:
import os, dotenv,sys
dotenv.load_dotenv(override=True)

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))


# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
# Azure Search
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
# Azure AI Document Intelligence
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_API_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_API_KEY")
# Azure Blob Storage
AZURE_STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
AZURE_STORAGE_CONTAINER = os.getenv("AZURE_STORAGE_CONTAINER")
AZURE_STORAGE_FOLDER = os.getenv("AZURE_STORAGE_FOLDER")
# SQL Database
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_DB = os.getenv("SQL_DB")
SQL_USERNAME = os.getenv("SQL_USERNAME")
SQL_PWD = os.getenv("SQL_PWD")

# Define the questions list (if you are using your own dataset you need to change this list)
QUESTIONS = [
  "What are the revenues of GOOGLE in the year 2009?",
  "What are the revenues and the operative margins of ALPHABET Inc. in 2022 and how it compares with the previous year?",
  "Can you create a table with the total revenue for ALPHABET, NVIDIA, MICROSOFT and APPLE in year 2023?",
  "Can you give me the Fiscal Year 2023 Highlights for APPLE, MICROSOFT and NVIDIA?",
  "Did APPLE repurchase common stock in 2023? create a table of APPLE repurchased stock with date, numbers of stocks and values in dollars.",
  "What is the value of the cumulative 5-years total return of ALPHABET Class A at December 2022?",
  "What was the price of APPLE, NVIDIA and MICROSOFT stock in 23/07/2024?",
  "Can you buy 10 shares of APPLE for me?"
  ]

# Define the System prompt (you need to update this is you are using your own dataset.)
system_prompt_RAG = """ You are a financial assistant tasked with answering questions related to the financial results of major technology companies listed on NASDAQ, \n
specifically Microsoft (MSFT), Alphabet Inc. (GOOGL), Nvidia (NVDA), Apple Inc. (AAPL), and Amazon (AMZN). \n
if you don't find the answer in the context, just say `I don't know.`"""

system_prompt_START = """
  You are an agent that needs analyze the user question. \n
  Question : {input} \n
  if the question is related to stock prices answer with "stock". \n
  if the question is related to information about financial results answer with "rag". \n
  if the question is unclear or you cannot decide answer with "rag". \n
  only answer with one of the word provided.
  Your answer (stock/rag):
  """
system_prompt_SQL = """
  You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about stock prices.
  If you can't find the answer, say 'I am unable to find the answer.'
  """


## Step 1: Import data to the SQL Database

The database provided is an Azure SQL Database.
Upload the data to the SQL Database using the `./data/fsi/db/create_stock_table.sql` script.

The script will create a table named "stock" in the provided database.
The table contains the following columns:
- **Date** DATE
- **CloseLast** DECIMAL(10, 2) 
- **Volume** INT
- **Open** DECIMAL(10, 2)
- **High** DECIMAL(10, 2)
- **Low** DECIMAL(10, 2)
- **Symbol** VARCHAR(10)

**NOTE**: Be sure that your IP address is not allowed to access the server.  To enable access, use the Azure Management Portal.
It may take up to five minutes for this change to take effect

<div class="alert alert-block alert-warning">
After the Step will be completed, you don't need to run this anymore. it's a one time step as the SQL Table has been already populated. 
</div>

In [ ]:
# Create the Connection to the SQL Server Database (sqlalchemy)
engine = create_engine(f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")

In [ ]:
# Create the Stock Table in the SQL Server Database
with open('../../data/fsi/db/create_stock_table.sql', 'r') as file:
    sql_statements = file.read()

# Execute the SQL Statements
with engine.connect() as connection:
    for command in sql_statements.split('GO\n'):
        command = command.strip()
        if command:
            connection.execute(text(command))
    connection.execute(text("commit"))

## Step 2: Create a LangChain SQL Agent

In [ ]:
# Create the Azure OpenAI Chat Client
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

In [ ]:
# Create the LangChain SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
# Create the Connection to the SQL Server Database (sqlalchemy)
engine = create_engine(f"mssql+pymssql://{SQL_USERNAME}:{SQL_PWD}@{SQL_SERVER}:1433/{SQL_DB}")


In [ ]:
# Create the stock_agent using the Langhcain SQL Agent Class (create_sql_agent)
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# Create the SQLDatabase object
db = SQLDatabase(engine)

# Create the SQLDatabaseToolkit object
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the stock_agent using create_sql_agent
stock_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
# Structure the final prompt from the ChatPromptTemplate
# Include the system message (provided at the biginning of the chapter - system_prompt_SQL) and the user message

from langchain.prompts.chat import ChatPromptTemplate

prompt_sql = ChatPromptTemplate.from_messages([
    ("system", system_prompt_SQL),
    ("human", "{question}")
])

In [ ]:
# Test the agent
print(QUESTIONS[6])
response = stock_agent.invoke(prompt_sql.format(question=QUESTIONS[6]))
print(response['output'])

## Step 3: Create the Multi Agent Solution

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../lib')))

## Import the necessary libraries
from typing import Annotated, Sequence
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import AzureAISearchRetriever
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langgraph.graph import StateGraph, END
from its_a_rag import ingestion

In [ ]:
# Create the Agent State Class to store the messages between the agents
# this should include the input, output and decision as strings
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
# Create the start_agent that analyze the user question and decide if the question is related to stock prices or financial results
def start_agent_node(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm, system_prompt_START

    # Prepare the prompt for the agent
    # Prompt Example: You are an agent that needs analyze the user question. \n Question : {input} \n if the question is related to stock prices answer with "stock". \n if the question is related to information about financial results answer with "rag". \n if the question is unclear or you cannot decide answer with "rag". \n only answer with one of the word provided. Your answer (stock/rag):

    from langchain.prompts import ChatPromptTemplate

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt_START),
        ("human", "{input}")
    ])

    # Prepare the chain to be executed
    formatted_prompt = prompt.format(input=state["input"])

    # invoke the chain
    response = llm.invoke(formatted_prompt)

    # take the decision from the response
    decision = response.content.strip().lower()
    if decision not in ["stock", "rag"]:
        decision = "rag"  # fallback to rag if unclear

    # Return the response for the next agent (decision and input required coming fron the Agent State)
    return {
        "input": state["input"],
        "decision": decision,
        "output": "",
        "messages": state.get("messages", [])
    }

In [ ]:
## Stock Agent

def stock_agent_node(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm, db, toolkit, stock_agent, prompt_sql

    # Create the SQL Database Object and the SQL Database Toolkit Object to be used by the agent.
    # (Already created globally: db, toolkit, stock_agent)

    # Create the agent using the Langhcain SQL Agent Class (create_sql_agent)
    # (Already created globally: stock_agent)

    # Structure the final prompt from the ChatPromptTemplate
    question = state["input"]
    formatted_prompt = prompt_sql.format(question=question)

    # Prepare the response using the invoke method of the agent
    response = stock_agent.invoke(formatted_prompt)

    # Return the response for the next agent (output and input required coming fron the Agent State)
    return {
        "input": state["input"],
        "decision": state.get("decision", "stock"),
        "output": response['output'],
        "messages": state.get("messages", [])
    }

In [ ]:
## Node rag (this is a clean implementation of the RAG Agent completed in Challenge 4)
def rag_agent_node(state):
    # Import the LLM (you can use "global" to import the LLM in previous step to avoid re-creating the LLM objects)
    global llm, system_prompt_RAG, AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_API_KEY, AZURE_SEARCH_INDEX

    # Define the index (use the one created in the previous challenge)
    retriever = AzureAISearchRetriever(
        api_key=AZURE_SEARCH_API_KEY,
        service_name=AZURE_SEARCH_ENDPOINT,
        index_name=AZURE_SEARCH_INDEX,
        top_k=20
    )

    # Define the chain (as it was in the previous challenge)
    # Step 1: Retrieve relevant documents and extract image descriptions/texts
    def retrieve_and_describe(query):
        docs = retriever.get_relevant_documents(query)
        context = ingestion.get_image_description(docs)
        return {"context": context, "question": query}

    # Step 2: Generate the multimodal prompt
    def build_multimodal_prompt(inputs):
        return ingestion.multimodal_prompt(inputs)

    # Step 3: Build the Langchain chain
    chain_multimodal_rag = (
        RunnableLambda(retrieve_and_describe)  # Retrieve docs and extract context
        | RunnableLambda(build_multimodal_prompt)  # Build prompt with system, context, question
        | llm  # Azure OpenAI Chat client
        | StrOutputParser()  # Parse output
    )

    
    # prepare the response using the invoke method of the agent
    response = chain_multimodal_rag.invoke(state["input"])

    # Return the response for the next agent (output and input required coming from the Agent State)
    return {
        "input": state["input"],
        "decision": state.get("decision", "rag"),
        "output": response,
        "messages": state.get("messages", [])
    }



In [ ]:
# Create the 3 steps graph that is going to be working in the bellow "decision" condition
# Add nodes (start_agent, stock_agent, rag_agent) and conditional edges where the decision with be stock or rag
def create_graph():
    # Create the Workflow as StateGraph using the AgentState
    workflow = StateGraph(AgentState)

    # Add the nodes (start_agent, stock_agent, rag_agent)
    workflow.add_node("start_agent", start_agent_node)
    workflow.add_node("stock_agent", stock_agent_node)
    workflow.add_node("rag_agent", rag_agent_node)

    # Add the conditional edge from start -> lambda (decision) -> stock_agent or rag_agent
    def route_decision(state):
        return state["decision"]

    workflow.add_conditional_edges(
        "start_agent",
        route_decision,
        {
            "stock": "stock_agent",
            "rag": "rag_agent"
        }
    )

    # Set the workflow entry point
    workflow.set_entry_point("start_agent")

    # Add the final edges to the END node
    workflow.add_edge("stock_agent", END)
    workflow.add_edge("rag_agent", END)

    # Compile and return the workflow
    return workflow.compile()

### Step 4: Test the Solution

In [ ]:
## Test Solution

# intantiate the graph (create_graph)
graph = create_graph()

# Use the graph invoke to answer the questions
# Test the graph with various questions

for QUESTION in QUESTIONS:
    print (QUESTION)
    result = graph.invoke({"input": QUESTION})
    print(result["output"])
    print ("------------------")